googleドライブ

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


huggingface

In [2]:
!pip install huggingface_hub

from google.colab import userdata
from huggingface_hub import login

# Secretsからトークンを取得
HF_TOKEN = userdata.get('HF_TOKEN')
login(HF_TOKEN)

ライブラリ

In [2]:
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --upgrade torch
!pip install --upgrade xformers

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

Found existing installation: unsloth 2025.3.18
Uninstalling unsloth-2025.3.18:
  Successfully uninstalled unsloth-2025.3.18
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-nhzjub2o/unsloth_bac65cdd707b45dd95717d7abb14b32e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-nhzjub2o/unsloth_bac65cdd707b45dd95717d7abb14b32e
  Resolved https://github.com/unslothai/unsloth.git to commit e80d642bc777f7a219bdd34aea1a77751f066785
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 272.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 331.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 307.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 327.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

パラメータ

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from peft import PeftModel
import torch
import json
from tqdm import tqdm
import re
import gc

# モデルとアダプターのパス
model_id = "/content/drive/MyDrive/program/code_translation/base_model/llm-jp-3-1.8b"
model_adapter = "test_1"
adapter_id = f"/content/drive/MyDrive/program/code_translation/model_adapter/{model_adapter}"

# ベンチマークデータのパス
benchmark_data = "/content/drive/MyDrive/program/code_translation/data/elyza-tasks-100-TV_test.jsonl"

# モデルの設定
dtype = None  # Noneにしておけば自動で設定
load_in_4bit = True  # 今回は1.8Bモデルを扱うためTrue

# モデルとトークナイザーの読み込み
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    trust_remote_code=True,
)

# アダプターの適用
model = PeftModel.from_pretrained(model, adapter_id)

# LLaMA-3.1のチャットテンプレートを適用
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# データセットの読み込み
datasets = []
with open(benchmark_data, "r") as f:
    item = ""
    for line in f:
        line = line.strip()
        item += line
        if item.endswith("}"):
            datasets.append(json.loads(item))
            item = ""

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


推論

In [ ]:
# 推論モードに切り替え
FastLanguageModel.for_inference(model)

# 推論の実行
results = []
for dt in tqdm(datasets):
    input_text = dt["input"]

    # チャット形式のメッセージを作成
    messages = [
        {"role": "user", "content": f"### 指示\n{input_text}\n### 回答\n"}
    ]

    # チャットテンプレートを適用してトークナイズ
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,  # 生成用に必要
        return_tensors="pt",
    ).to(model.device)

    # モデルによる生成
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        use_cache=True,
        do_sample=False,  # サンプリングなし
        repetition_penalty=1.2,
    )

    # 出力をデコード
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### 回答')[-1].strip()

    # 結果を保存
    results.append({"task_id": dt["task_id"], "input": input_text, "output": prediction})



100%|██████████| 10/10 [02:32<00:00, 15.24s/it]


保存

In [ ]:
# 結果をjsonlで保存
json_file_id = re.sub(".*/", "", adapter_id)

# ローカルに保存
with open(f"/content/{json_file_id}_output.jsonl", 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')

# Google Driveに保存
with open(f"/content/drive/MyDrive/program/code_translation/data/{json_file_id}_output.jsonl", 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')